In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext line_profiler

In [3]:
import itertools
import torch

from tsvar.simulate import GrangerBuscaSimulator
from tsvar.wold_model import VariationalWoldModel

---

## Generate small toy example dataset

Define model parameters

In [4]:
dim = 3

baseline = np.array([0.1, 0.1, 0.2])

# adjacency[i,j] = magnitude of influence from i to j
adjacency = np.array([
    [0.3, 0.8, 0.1],
    [0.2, 0.3, 0.1],
    [0.2, 0.1, 0.0]
])

beta = 1.0

end_time = 1e5

for i, j in itertools.product(range(dim), repeat=2):
    print(f"w({j:d} -> {i:d}) = {adjacency[j,i]:.2f}")

w(0 -> 0) = 0.30
w(1 -> 0) = 0.20
w(2 -> 0) = 0.20
w(0 -> 1) = 0.80
w(1 -> 1) = 0.30
w(2 -> 1) = 0.10
w(0 -> 2) = 0.10
w(1 -> 2) = 0.10
w(2 -> 2) = 0.00


Simulate a dataset

In [5]:
wold_sim = GrangerBuscaSimulator(mu_rates=baseline, Alpha_ba=adjacency, Beta_b=beta * np.ones((dim,)))
events = wold_sim.simulate(end_time, seed=42)
events = [torch.tensor(ev, dtype=torch.float) for ev in events]


print(list(map(len, events)))
print()
print(events[0])
print()
print(events[1])
print()
print(events[2])

[36694, 55152, 28247]

tensor([1.7752e+01, 2.5812e+01, 2.6760e+01,  ..., 9.9980e+04, 9.9987e+04,
        9.9997e+04])

tensor([3.8479e-01, 8.3199e+00, 2.4794e+01,  ..., 9.9994e+04, 9.9998e+04,
        9.9999e+04])

tensor([1.8646e+00, 1.4326e+01, 1.9811e+01,  ..., 9.9965e+04, 9.9972e+04,
        9.9988e+04])


In [6]:
sum(map(len, events))

120093

---

## Test variational inference algorithm

Create model object and set the data.

In [7]:
self = VariationalWoldModel(verbose=True)
self.set_data(events)

Define the parameters of the prior.

In [8]:
as_pr = 1.0 * np.ones((dim + 1, dim))
ar_pr = 1.0 * np.ones((dim + 1, dim))

bs_pr = 10.0 * np.ones((dim, dim))
br_pr = 10.0 * np.ones((dim, dim))

zc_pr = [1.0 * np.ones((len(events[i]), dim+1)) for i in range(dim)]

In [9]:
self.fit(as_pr=as_pr, ar_pr=ar_pr, bs_pr=bs_pr, br_pr=br_pr, zc_pr=zc_pr, max_iter=100, tol=1e-5)

-------------------------------------------------- 0
Alpha posterior mean:
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
Z[0] posterior probabilities
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 ...
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
-------------------------------------------------- 1
Alpha posterior mean:
[[ 9174.5  13789.    7062.75]
 [    0.09     0.14     0.07]
 [    0.09     0.14     0.07]
 [    0.09     0.14     0.07]]
Z posterior probabilities
[[0.99466 0.00154 0.00054 0.00326]
 [0.      0.      0.99998 0.00002]
 [0.      0.9642  0.      0.0358 ]
 ...
 [0.      0.      0.      1.     ]
 [0.      0.      0.      1.     ]
 [0.      0.      0.      1.     ]]
-------------------------------------------------- 2
Alpha posterior mean:
[[4736.2  7982.73 2967.34]
 [   0.38    0.42    0.31]
 [   0.1     0.12    0.08]
 [   0.79    1.36    0.66]]
Z posterior probabilities
[[0.02922 0.24159 0.03796 0.69123]
 [0.      0.      1.

Alpha posterior mean:
[[ 1.    1.    1.  ]
 [13.84 17.5  10.93]
 [ 6.54  8.58  4.98]
 [20.76 34.34 17.24]]
Z posterior probabilities
[[0.      0.01965 0.      0.98035]
 [0.      0.      1.      0.     ]
 [0.      1.      0.      0.     ]
 ...
 [0.      0.      0.      1.     ]
 [0.      0.      0.      1.     ]
 [0.      0.      0.      1.     ]]
-------------------------------------------------- 25
Alpha posterior mean:
[[ 1.    1.    1.  ]
 [14.43 18.24 11.39]
 [ 6.81  8.93  5.18]
 [21.65 35.81 17.98]]
Z posterior probabilities
[[0.      0.01686 0.      0.98314]
 [0.      0.      1.      0.     ]
 [0.      1.      0.      0.     ]
 ...
 [0.      0.      0.      1.     ]
 [0.      0.      0.      1.     ]
 [0.      0.      0.      1.     ]]
-------------------------------------------------- 26
Alpha posterior mean:
[[ 1.    1.    1.  ]
 [15.01 18.98 11.85]
 [ 7.08  9.29  5.39]
 [22.53 37.28 18.71]]
Z posterior probabilities
[[0.      0.01446 0.      0.98554]
 [0.      0.      1.      

 [0.      0.      0.      1.     ]]
-------------------------------------------------- 47
Alpha posterior mean:
[[ 1.    1.    1.  ]
 [26.83 34.   21.15]
 [12.5  16.49  9.44]
 [40.4  66.91 33.48]]
Z posterior probabilities
[[0.      0.00057 0.      0.99943]
 [0.      0.      1.      0.     ]
 [0.      1.      0.      0.     ]
 ...
 [0.      0.      0.      1.     ]
 [0.      0.      0.      1.     ]
 [0.      0.      0.      1.     ]]
-------------------------------------------------- 48
Alpha posterior mean:
[[ 1.    1.    1.  ]
 [27.37 34.69 21.57]
 [12.74 16.82  9.62]
 [41.22 68.26 34.15]]
Z posterior probabilities
[[0.      0.00049 0.      0.99951]
 [0.      0.      1.      0.     ]
 [0.      1.      0.      0.     ]
 ...
 [0.      0.      0.      1.     ]
 [0.      0.      0.      1.     ]
 [0.      0.      0.      1.     ]]
-------------------------------------------------- 49
Alpha posterior mean:
[[ 1.    1.    1.  ]
 [27.9  35.37 21.99]
 [12.99 17.14  9.8 ]
 [42.02 69.6  34.82

Z posterior probabilities
[[0.      0.00003 0.      0.99997]
 [0.      0.      1.      0.     ]
 [0.      1.      0.      0.     ]
 ...
 [0.      0.      0.      1.     ]
 [0.      0.      0.      1.     ]
 [0.      0.      0.      1.     ]]
-------------------------------------------------- 68
Alpha posterior mean:
[[ 1.    1.    1.  ]
 [37.45 47.67 29.47]
 [17.33 22.99 12.99]
 [56.5  93.64 46.66]]
Z posterior probabilities
[[0.      0.00003 0.      0.99997]
 [0.      0.      1.      0.     ]
 [0.      1.      0.      0.     ]
 ...
 [0.      0.      0.      1.     ]
 [0.      0.      0.      1.     ]
 [0.      0.      0.      1.     ]]
-------------------------------------------------- 69
Alpha posterior mean:
[[ 1.    1.    1.  ]
 [37.92 48.29 29.84]
 [17.55 23.28 13.15]
 [57.22 94.83 47.24]]
Z posterior probabilities
[[0.      0.00002 0.      0.99998]
 [0.      0.      1.      0.     ]
 [0.      1.      0.      0.     ]
 ...
 [0.      0.      0.      1.     ]
 [0.      0.      0.   

-------------------------------------------------- 92
Alpha posterior mean:
[[  1.     1.     1.  ]
 [ 47.82  61.31  37.6 ]
 [ 22.08  29.46  16.4 ]
 [ 72.26 119.82  59.39]]
Z posterior probabilities
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
-------------------------------------------------- 93
Alpha posterior mean:
[[  1.     1.     1.  ]
 [ 48.21  61.83  37.9 ]
 [ 22.26  29.7   16.53]
 [ 72.85 120.8   59.87]]
Z posterior probabilities
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
-------------------------------------------------- 94
Alpha posterior mean:
[[  1.     1.     1.  ]
 [ 48.6   62.34  38.21]
 [ 22.44  29.95  16.66]
 [ 73.44 121.77  60.34]]
Z posterior probabilities
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
-------------------------------------------------- 95
Alpha posterior mean:
[[  1.     1.     1.  ]
 [ 48.98  62.85

In [10]:
alpha_hat_mean = np.round(self._as_po / self._ar_po, 2)
baseline_hat = alpha_hat_mean[0,:]
adjacency_hat = alpha_hat_mean[1:,:]
beta_hat = np.round(self._br_po / (self._bs_po - 1), 2)

print('Baseline:')
print('---------')
print('Ground truth:')
print(baseline)
print('Estimated:')
print(baseline_hat)
print()

print('Adjacency:')
print('---------')
print('Ground truth:')
print(adjacency)
print('Estimated:')
print(adjacency_hat)
print()

print('Beta:')
print('-----')
print('Ground truth:')
print(beta * np.ones((dim, dim)))
print('Estimated:')
print(beta_hat)

Baseline:
---------
Ground truth:
[0.1 0.1 0.2]
Estimated:
[1. 1. 1.]

Adjacency:
---------
Ground truth:
[[0.3 0.8 0.1]
 [0.2 0.3 0.1]
 [0.2 0.1 0. ]]
Estimated:
[[ 50.85  65.35  39.96]
 [ 23.48  31.37  17.4 ]
 [ 76.85 127.44  63.06]]

Beta:
-----
Ground truth:
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
Estimated:
[[417.78 394.39 422.35]
 [372.23 366.8  375.12]
 [431.66 433.41 455.86]]


In [11]:
self._as_po

array([[    1.     ,     1.     ,     1.     ],
       [12310.77795, 16759.19245,  9568.54467],
       [ 6377.24654,  8648.78427,  4688.94063],
       [18008.97551, 29747.02328, 13992.5147 ]])

In [12]:
self._ar_po

array([[  1.     ,   1.     ,   1.     ],
       [242.09323, 256.43993, 239.43734],
       [271.61426, 275.65851, 269.47064],
       [234.33698, 233.41334, 221.89112]])

In [13]:
self._bs_po

array([[12319.77795, 16768.19245,  9577.54467],
       [ 6386.24654,  8657.78427,  4697.94063],
       [18017.97551, 29756.02328, 14001.5147 ]])

In [14]:
self._br_po

array([[ 5146541.31313,  6612884.86112,  4044674.35605],
       [ 2376792.75805,  3175343.1271 ,  1761893.37873],
       [ 7777215.86217, 12896263.75717,  6382304.14974]])

In [22]:
self._zp_po[0].mean(axis=0)

array([0.     , 0.33547, 0.17377, 0.49076])

---

In [15]:
from scipy.special import digamma

i = 0

as_po=self._as_po
ar_po=self._ar_po
bs_po=self._bs_po
br_po=self._br_po
delta_ikj=self.delta_ikj
last_t = self.last_t
zp_po=self._zp_po

In [16]:
zp_po[i].shape

(36694, 4)

In [17]:
br_po / (bs_po - 1)

array([[417.78018, 394.39428, 422.35216],
       [372.23195, 366.804  , 375.11511],
       [431.66046, 433.41468, 455.86211]])

In [18]:
bs_po

array([[12319.77795, 16768.19245,  9577.54467],
       [ 6386.24654,  8657.78427,  4697.94063],
       [18017.97551, 29756.02328, 14001.5147 ]])